In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

In [2]:
df_air_reserve =       pd.read_csv("../data/raw/air_reserve.csv", parse_dates=['visit_datetime', 'reserve_datetime'])
df_air_store_info =    pd.read_csv("../data/raw/air_store_info.csv")
df_air_visit_data =    pd.read_csv("../data/raw/air_visit_data.csv", parse_dates=['visit_date'])

df_hpg_reserve =       pd.read_csv("../data/raw/hpg_reserve.csv", parse_dates=['visit_datetime', 'reserve_datetime'])
df_hpg_store_info =    pd.read_csv("../data/raw/hpg_store_info.csv")

df_date_info =         pd.read_csv("../data/raw/date_info.csv", parse_dates=['calendar_date'])
df_store_id_relation = pd.read_csv("../data/raw/store_id_relation.csv")

### CHECK DATASET SIZES

In [3]:
print('"df_air_reserve" dataset has %s rows' % df_air_reserve.shape[0])
print('"df_air_store_info" dataset has %s rows' % df_air_store_info.shape[0])
print('"df_air_visit_data" dataset has %s rows' % df_air_visit_data.shape[0])
print('')
print('"df_hpg_reserve" dataset has %s rows' % df_hpg_reserve.shape[0])
print('"df_hpg_store_info" dataset has %s rows' % df_hpg_store_info.shape[0])

"df_air_reserve" dataset has 92378 rows
"df_air_store_info" dataset has 829 rows
"df_air_visit_data" dataset has 252108 rows

"df_hpg_reserve" dataset has 2000320 rows
"df_hpg_store_info" dataset has 4690 rows


### INFO

There are significant number of records, especially in the 'df_air_visit_data' and 'df_hpg_reserve' logs. Merging/joining all dataset into one would result in the memory consumption over 2.4GB.

In order to speed up analysis and reduce utilization of resources (CPU, memory), the first approach of the experiment will be conducted only for the TOP 20 restaurants.

**I define TOP 20 restaurants as restaurants most VISITED by (hungry &#x1F642;) people.** 

### IDENTIFICATION OF TOP 20 RESTAURANTS

In [4]:
# The restaurants sorted by overall number of visitors
df_air_top20 = df_air_visit_data.groupby('air_store_id')['visitors'] \
                                .agg({"Overall number of visitors": "sum"}) \
                                .sort_values(by="Overall number of visitors", ascending=False) \
                                .head(20)

In [5]:
df_air_top20

,Overall number of visitors
air_store_id,
air_399904bdb7685ca0,18717
air_f26f36ec4dc5adb0,18577
air_e55abd740f93ecc4,18101
air_99157b6163835eec,18097
air_5c817ef28f236bdf,18009
air_d97dabf7aae60da5,17971
air_3525f11ef0bf0c35,17958
air_681f96e6a6595f82,17559
air_d34c0861a2be94cb,17467


## DATASET MERGING

In [6]:
# first, create dataset with air_store_id column only
df_top20_ids = pd.DataFrame(df_air_top20.index)

### RESTAURANT DETAILS

In [7]:
# create dataset with details of top 20 restaurants
df_top20_ids_info = pd.merge(df_top20_ids, 
                             df_air_store_info, 
                             how='left',
                             on=['air_store_id'])

In [8]:
df_top20_ids_info

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801
1,air_f26f36ec4dc5adb0,Izakaya,Tōkyō-to Shinjuku-ku Kabukichō,35.693840,139.703549
2,air_e55abd740f93ecc4,Izakaya,Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,34.756950,134.841177
3,air_99157b6163835eec,Cafe/Sweets,Shizuoka-ken Numazu-shi Miyukichō,35.095584,138.863493
4,air_5c817ef28f236bdf,Izakaya,Tōkyō-to Shibuya-ku Shibuya,35.661777,139.704051
5,air_d97dabf7aae60da5,Cafe/Sweets,Tōkyō-to Shibuya-ku Jingūmae,35.669290,139.707056
6,air_3525f11ef0bf0c35,Cafe/Sweets,Ōsaka-fu Ōsaka-shi Shinmachi,34.676231,135.486059
7,air_681f96e6a6595f82,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599
8,air_d34c0861a2be94cb,Cafe/Sweets,Fukuoka-ken Fukuoka-shi Momochi,33.581941,130.348436
9,air_082908692355165e,Italian/French,Tōkyō-to Shinjuku-ku Kabukichō,35.693840,139.703549


### ... WITH VISITS

In [9]:
df_top20_ids_info_visits = pd.merge(df_top20_ids_info, 
                                    df_air_visit_data, 
                                    how='left',
                                    on=['air_store_id'])

In [10]:
df_top20_ids_info_visits.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,visit_date,visitors
0,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-05,4
1,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-06,15
2,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-07,26
3,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-08,75
4,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-09,91


In [11]:
df_top20_ids_info_visits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8763 entries, 0 to 8762
Data columns (total 7 columns):
air_store_id      8763 non-null object
air_genre_name    8763 non-null object
air_area_name     8763 non-null object
latitude          8763 non-null float64
longitude         8763 non-null float64
visit_date        8763 non-null datetime64[ns]
visitors          8763 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 547.7+ KB


#### (PREPARE RESERVATIONS DATAFRAME)

In [12]:
df_air_reserve['visit_date'] = df_air_reserve['visit_datetime'].dt.date
df_air_reserve['visit_date'] = pd.to_datetime(df_air_reserve['visit_date'])

df_air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,2016-01-01
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,2016-01-01
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,2016-01-01
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,2016-01-01
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,2016-01-01


In [13]:
df_air_reserve.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92378 entries, 0 to 92377
Data columns (total 5 columns):
air_store_id        92378 non-null object
visit_datetime      92378 non-null datetime64[ns]
reserve_datetime    92378 non-null datetime64[ns]
reserve_visitors    92378 non-null int64
visit_date          92378 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(1)
memory usage: 3.5+ MB


In [14]:
# The restaurants sorted by overall number of reserve visitors
df1 = df_air_reserve.groupby(['air_store_id','visit_date'])['reserve_visitors'] \
                    .agg({"reserve_visitors_sum": "sum"}) 
    
df_reservation_data = pd.DataFrame({'air_reserve_visitors_sum' : df1['reserve_visitors_sum']}).reset_index()
df_reservation_data.head()

,air_store_id,visit_date,air_reserve_visitors_sum
0,air_00a91d42b08b08d9,2016-10-31,2
1,air_00a91d42b08b08d9,2016-12-05,9
2,air_00a91d42b08b08d9,2016-12-14,18
3,air_00a91d42b08b08d9,2016-12-17,2
4,air_00a91d42b08b08d9,2016-12-20,4


In [15]:
df_reservation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29830 entries, 0 to 29829
Data columns (total 3 columns):
air_store_id                29830 non-null object
visit_date                  29830 non-null datetime64[ns]
air_reserve_visitors_sum    29830 non-null int64
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 699.2+ KB


### ... WITH RESERVATIONS

In [16]:
df_top20_ids_info_visits_reservations = pd.merge(df_top20_ids_info_visits, 
                                                 df_reservation_data, 
                                                 how='left',
                                                 on=['air_store_id','visit_date'])

In [17]:
df_top20_ids_info_visits_reservations.head(5)

,air_store_id,air_genre_name,air_area_name,latitude,longitude,visit_date,visitors,air_reserve_visitors_sum
0,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-05,4,NaN
1,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-06,15,NaN
2,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-07,26,NaN
3,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-08,75,NaN
4,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-09,91,NaN


In [18]:
df_top20_ids_info_visits_reservations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8763 entries, 0 to 8762
Data columns (total 8 columns):
air_store_id                8763 non-null object
air_genre_name              8763 non-null object
air_area_name               8763 non-null object
latitude                    8763 non-null float64
longitude                   8763 non-null float64
visit_date                  8763 non-null datetime64[ns]
visitors                    8763 non-null int64
air_reserve_visitors_sum    1513 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 616.1+ KB


#### Post-merge data verification...

The last column `reserve_visitors_sum` contains some NaN values so I need to investigate why this happens.

In [19]:
# Lets take first 'air_store_id' and first 'visit_date' 
# for whom the df_top20_ids_info_visits_reservations['reserve_visitors_sum'] is NaN.
df_top20_ids_info_visits_reservations[ (df_top20_ids_info_visits_reservations['air_store_id'] == 'air_399904bdb7685ca0') & \
                                       (df_top20_ids_info_visits_reservations['visit_date'] == '2016-01-05')]

,air_store_id,air_genre_name,air_area_name,latitude,longitude,visit_date,visitors,air_reserve_visitors_sum
0,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-05,4,NaN


In [20]:
# Let's see if we have any reservation record for the restaurant on particular day
df_reservation_data[ (df_reservation_data['air_store_id'] == 'air_399904bdb7685ca0') & \
                     (df_reservation_data['visit_date'] == '2016-01-05')]

,air_store_id,visit_date,air_reserve_visitors_sum


In [21]:
# Let's check earliest reservations records for the restaurant
df_reservation_data[(df_reservation_data['air_store_id'] == 'air_399904bdb7685ca0')] \
    .sort_values(by=['visit_date'], ascending=[1]) \
    .head(5)

,air_store_id,visit_date,air_reserve_visitors_sum
7469,air_399904bdb7685ca0,2016-10-27,15
7470,air_399904bdb7685ca0,2016-10-28,17
7471,air_399904bdb7685ca0,2016-10-31,10
7472,air_399904bdb7685ca0,2016-11-04,5
7473,air_399904bdb7685ca0,2016-11-06,10


In [22]:
df_air_reserve[df_air_reserve['air_store_id'] == 'air_399904bdb7685ca0'].head(5)

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date
16446,air_399904bdb7685ca0,2016-10-27 21:00:00,2016-10-27 16:00:00,4,2016-10-27
16447,air_399904bdb7685ca0,2016-10-27 21:00:00,2016-10-21 21:00:00,11,2016-10-27
16573,air_399904bdb7685ca0,2016-10-28 18:00:00,2016-10-27 20:00:00,2,2016-10-28
16757,air_399904bdb7685ca0,2016-10-28 19:00:00,2016-10-27 23:00:00,2,2016-10-28
16860,air_399904bdb7685ca0,2016-10-28 20:00:00,2016-10-27 16:00:00,4,2016-10-28


Apparently, the NaN values in the `df_top20_ids_info_visits_reservations['reserve_visitors_sum']` column dataframe are caused by missing data in the provided data.

### HPG

The HPG data will be only partially included in the dataset for my experiment.

Reasons:
* The submission file contains only `air_store_id ` and does not contain information about `hpg_store_id`.
* The HPG data provide only information about reservations, they do not provide information about visitors that really have come to restaurants.
* My first attempt will initially focus on predicting number of visitors

#### (PREPARE RESERVATIONS DATAFRAME)

In [23]:
df_hpg_reserve.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000320 entries, 0 to 2000319
Data columns (total 4 columns):
hpg_store_id        object
visit_datetime      datetime64[ns]
reserve_datetime    datetime64[ns]
reserve_visitors    int64
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 61.0+ MB


In [24]:
df_hpg_reserve['visit_date'] = df_hpg_reserve['visit_datetime'].dt.date
df_hpg_reserve['visit_date'] = pd.to_datetime(df_hpg_reserve['visit_date'])

df_hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,visit_date
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1,2016-01-01
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3,2016-01-01
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2,2016-01-01
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5,2016-01-01
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13,2016-01-01


In [25]:
# The restaurants sorted by overall number of reserve visitors
df2 = df_hpg_reserve.groupby(['hpg_store_id','visit_date'])['reserve_visitors'] \
                    .agg({"reserve_visitors_sum": "sum"}) 
    
df_hpg_reservation_data = pd.DataFrame({'hpg_reserve_visitors_sum' : df2['reserve_visitors_sum']}).reset_index()
df_hpg_reservation_data.head()

,hpg_store_id,visit_date,hpg_reserve_visitors_sum
0,hpg_001112ef76b9802c,2016-02-26,9
1,hpg_001112ef76b9802c,2016-03-17,3
2,hpg_001112ef76b9802c,2016-03-31,5
3,hpg_001112ef76b9802c,2016-04-05,13
4,hpg_001112ef76b9802c,2016-04-18,9


In [26]:
# for brevity I'll introduce new name
df_top20 = df_top20_ids_info_visits_reservations

### ... WITH HPG

In [27]:
df_top20_hpg = pd.merge(df_top20, 
                        df_store_id_relation, 
                        how='left',
                        on=['air_store_id'])

In [28]:
df_top20_hpg.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,visit_date,visitors,air_reserve_visitors_sum,hpg_store_id
0,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-05,4,NaN,NaN
1,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-06,15,NaN,NaN
2,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-07,26,NaN,NaN
3,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-08,75,NaN,NaN
4,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-09,91,NaN,NaN


### ... WITH HPG RESERVATIONS

In [29]:
df_top20_hpg_reservations = pd.merge(df_top20_hpg, 
                                     df_hpg_reservation_data, 
                                     how='left',
                                     on=['hpg_store_id', 'visit_date'])

In [30]:
df_top20_hpg_reservations.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,visit_date,visitors,air_reserve_visitors_sum,hpg_store_id,hpg_reserve_visitors_sum
0,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-05,4,NaN,NaN,NaN
1,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-06,15,NaN,NaN,NaN
2,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-07,26,NaN,NaN,NaN
3,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-08,75,NaN,NaN,NaN
4,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-09,91,NaN,NaN,NaN


### ... WITH DATE INFO

In [31]:
df_date_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 3 columns):
calendar_date    517 non-null datetime64[ns]
day_of_week      517 non-null object
holiday_flg      517 non-null int64
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 12.2+ KB


In [32]:
df_top20_hpg_reservations_dateinfo = pd.merge(df_top20_hpg_reservations, 
                                              df_date_info, 
                                              how='left',
                                              left_on='visit_date',
                                              right_on='calendar_date')

In [33]:
df_top20_hpg_reservations_dateinfo.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,visit_date,visitors,air_reserve_visitors_sum,hpg_store_id,hpg_reserve_visitors_sum,calendar_date,day_of_week,holiday_flg
0,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-05,4,NaN,NaN,NaN,2016-01-05,Tuesday,0
1,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-06,15,NaN,NaN,NaN,2016-01-06,Wednesday,0
2,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-07,26,NaN,NaN,NaN,2016-01-07,Thursday,0
3,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-08,75,NaN,NaN,NaN,2016-01-08,Friday,0
4,air_399904bdb7685ca0,Italian/French,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,34.681261,135.509801,2016-01-09,91,NaN,NaN,NaN,2016-01-09,Saturday,0


In [34]:
# Save data in the file
df_top20_hpg_reservations_dateinfo.to_csv("../data/processed/restaurants_top20.csv", 
                                             sep=',', 
                                             encoding='utf-8', 
                                             index=False)

### Another dataset with all 'AIR' stores

In [35]:
# all stores
df_all = df_air_store_info

# with visits
df_all_visits = pd.merge(df_all, 
                         df_air_visit_data, 
                         how='left',
                         on=['air_store_id'])

# with reservations
df_all_visits_reservations = pd.merge(df_all_visits, 
                                      df_reservation_data, 
                                      how='left',
                                      on=['air_store_id','visit_date'])

df_all_visits_reservations_hpg = pd.merge(df_all_visits_reservations, 
                                          df_store_id_relation, 
                                          how='left',
                                          on=['air_store_id'])

# with hpg reservations
df_all_visits_reservations_hpg_reservations = pd.merge(df_all_visits_reservations_hpg, 
                                                       df_hpg_reservation_data, 
                                                       how='left',
                                                       on=['hpg_store_id', 'visit_date'])

# with date info
df_all_visits_reservations_hpg_reservations_dateinfo = pd.merge(df_all_visits_reservations_hpg_reservations, 
                                                                df_date_info, 
                                                                how='left',
                                                                left_on='visit_date',
                                                                right_on='calendar_date')

# save
df_all_visits_reservations_hpg_reservations_dateinfo.to_csv("../data/processed/restaurants_all.csv", 
                                                            sep=',', 
                                                            encoding='utf-8', 
                                                            index=False)

### verify columns in the saved datasets

In [36]:
df_top20_from_file = pd.read_csv("../data/processed/restaurants_top20.csv")
df_top20_from_file.columns

Index(['air_store_id', 'air_genre_name', 'air_area_name', 'latitude',
       'longitude', 'visit_date', 'visitors', 'air_reserve_visitors_sum',
       'hpg_store_id', 'hpg_reserve_visitors_sum', 'calendar_date',
       'day_of_week', 'holiday_flg'],
      dtype='object')

In [37]:
df_all_from_file = pd.read_csv("../data/processed/restaurants_all.csv")
df_all_from_file.columns

Index(['air_store_id', 'air_genre_name', 'air_area_name', 'latitude',
       'longitude', 'visit_date', 'visitors', 'air_reserve_visitors_sum',
       'hpg_store_id', 'hpg_reserve_visitors_sum', 'calendar_date',
       'day_of_week', 'holiday_flg'],
      dtype='object')

### info

In [38]:
df_top20_from_file.shape

(8763, 13)

In [39]:
df_all_from_file.shape

(252108, 13)